In [1]:
import os
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import lxml
from openpyxl import Workbook

# Final Project

## Youtube의 직접 만든 playlist 링크를 활용하여 영상 목록과 자세한 영상 정보(영상 제목, 좋아요 수, 크리에이터 이름, 구독자 수, 조회수)를 수집하여 엑셀 시트에 저장하세요. 

1.  Selenium Webdriver를 시작하세요

In [2]:
DRIVER_PATH = os.getcwd() + "\\msedgedriver.exe"

eg_driver = webdriver.Edge(DRIVER_PATH)

C:\Users\user\AppData\Local\Temp\ipykernel_3776\4132983183.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  eg_driver = webdriver.Edge(DRIVER_PATH)


2. 직접 만든 playlist의 uri를 playlist_uri에 할당하세요

In [3]:
playlist_uri = "https://www.youtube.com/playlist?list=PL-Rk9-7qrgwd2thtw0ZUjB5HMKv2kz9Zb"

3. Webdriver로 playlist_uri에 접속하세요

In [4]:
eg_driver.get(playlist_uri)

4. webdriver의 page_source를 BeautifulSoup으로 분석하여 목록의 제목과 링크를 구하세요 

In [5]:
BASE_URL = "https://www.youtube.com"
SEL_CONTENT = "#meta > h3 a"

soup = BeautifulSoup(eg_driver.page_source, 'lxml')

e_content = soup.select(SEL_CONTENT)

playlist_titles = [c.text.strip("\n ") for c in e_content[:-1]]
playlist_links = [BASE_URL + c['href'] for c in e_content[:-1]]

for t, l in zip(playlist_titles, playlist_links):
    print("{}: {}".format(t, l))

chatGPT는 엄준식 프로그래밍 언어도 잘할까: https://www.youtube.com/watch?v=SdDs0ScOFSU&list=PL-Rk9-7qrgwd2thtw0ZUjB5HMKv2kz9Zb&index=1
미친 파이썬 프레임워크가 나타났다.: https://www.youtube.com/watch?v=47BL6WLZJ1g&list=PL-Rk9-7qrgwd2thtw0ZUjB5HMKv2kz9Zb&index=2
코딩 빠르게 하는 꿀팁 및 설정 🍯 (자동 완성 기능): https://www.youtube.com/watch?v=umeqCopb96w&list=PL-Rk9-7qrgwd2thtw0ZUjB5HMKv2kz9Zb&index=3
위기: https://www.youtube.com/watch?v=DEZltZdk-o0&list=PL-Rk9-7qrgwd2thtw0ZUjB5HMKv2kz9Zb&index=4
한국에 집을 사는 대신, 뉴욕에서 월세살이를 시작한 부부 ep.1: https://www.youtube.com/watch?v=aX_Oan6KkFw&list=PL-Rk9-7qrgwd2thtw0ZUjB5HMKv2kz9Zb&index=5


5. 첫번째 링크로 영상의 페이지로 이동하세요

In [6]:
eg_driver.get(playlist_links[0])

6. 해당 페이지의 page_source를 BeautifulSoup으로 분석하여 영상 제목, 좋아요 수, 크리에이터 이름, 구독자 수, 조회수를 수집하세요

In [7]:
def get_video_info(agent, video_uri = None, wait_time = 10):

    PATH_TITLE = "#title > h1"
    PATH_CREATOR = "#text > a"
    PATH_SUB_COUNT = "#owner-sub-count"
    PATH_LIKE = "#segmented-like-button > ytd-toggle-button-renderer > yt-button-shape > button > div.cbox.yt-spec-button-shape-next--button-text-content > span"
    PATH_VIEW = "#info > span:nth-child(1)"

    def _stat_exchanger(c_unit: str) -> int:
        ABBR = {"k": 3, "m": 6, "b": 9, "t": 12, '천': 3, "만": 4, "억": 8, "조": 12}

        if c_unit.isnumeric():
            return int(c_unit)
        elif c_unit[-1].lower() in ABBR.keys():
            return int(float(c_unit[:-1]) * (10 ** ABBR[c_unit[-1].lower()]))
        else:
            return "Unit not found"

    if video_uri is not None:
        eg_driver.get(video_uri)   
        
    wait = WebDriverWait(agent, wait_time)

    video_title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, PATH_TITLE))).text
    
    creator_name = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, PATH_CREATOR))).text
    
    sub_count = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, PATH_SUB_COUNT))).text
    sub_count = sub_count.strip("구독자 명")
    sub_count = _stat_exchanger(sub_count)
    
    likes = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, PATH_LIKE))).text
    likes = _stat_exchanger(likes)

    views = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, PATH_VIEW))).text
    views = views.strip("조회수 회")
    views = _stat_exchanger(views)

    return video_title, creator_name, sub_count, likes, views

video_title, creator_name, sub_count, likes, views = get_video_info(eg_driver)

print("title: {},\nname: {},\nsub_count: {},\nlikes: {},\nviews: {}\n".format(video_title, creator_name, sub_count, likes, views))

title: chatGPT는 엄준식 프로그래밍 언어도 잘할까,
name: 코딩애플,
sub_count: 135000,
likes: 2400,
views: 59000



7. 5,6번의 수행과정을 모든 링크에 대해 반복수행하세요

In [8]:
video_title = []
creator_name = []
sub_count = []
likes = []
views = []

for uri in playlist_links:
    t, n, s, l, v = get_video_info(eg_driver, video_uri = uri, wait_time = 10)

    video_title.append(t)
    creator_name.append(n)
    sub_count.append(s)
    likes.append(l)
    views.append(v)

    print("title: {},\nname: {},\nsub_count: {},\nlikes: {},\nviews: {}\n".format(t, n, s, l, v))

title: chatGPT는 엄준식 프로그래밍 언어도 잘할까,
name: 코딩애플,
sub_count: 135000,
likes: 2400,
views: 59000

title: 미친 파이썬 프레임워크가 나타났다.,
name: 노마드 코더 Nomad Coders,
sub_count: 452000,
likes: 2600,
views: 120000

title: 코딩 빠르게 하는 꿀팁 및 설정 🍯 (자동 완성 기능),
name: 드림코딩,
sub_count: 161000,
likes: 687,
views: 18000

title: 위기,
name: 수빙수tv sooBingsoo,
sub_count: 1190000,
likes: 4100,
views: 270000

title: 한국에 집을 사는 대신, 뉴욕에서 월세살이를 시작한 부부 ep.1,
name: 유랑쓰 YOURANGSS,
sub_count: 123000,
likes: 6600,
views: 300000



8. 수집한 데이터를 엑셀 시트에 저장하세요.

In [9]:
FILENAME = "result.xlsx"

wb = Workbook()
ws = wb.active

column_names = ['Title', 'URL', 'Creator Name', 'Subscribers', 'Likes', 'Vies']

for j, col in enumerate(column_names):
    ws.cell(row = 1, column = j + 1, value = col)

for i in range(len(playlist_links)):
    ws.cell(row = 2 + i, column = 1, value = video_title[i])
    ws.cell(row = 2 + i, column = 2, value = playlist_links[i])
    ws.cell(row = 2 + i, column = 3, value = creator_name[i])
    ws.cell(row = 2 + i, column = 4, value = sub_count[i])
    ws.cell(row = 2 + i, column = 5, value = likes[i])
    ws.cell(row = 2 + i, column = 6, value = views[i])

wb.save('./' + FILENAME)
wb.close()

In [10]:
eg_driver.close()

# ----- 종합 (객체지향) -----

In [26]:
import os
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import lxml
from openpyxl import Workbook


class YoutubePlaylist(webdriver.Edge):
    
    URI_BASE = 'https://www.youtube.com'
    URI_PLAYLIST = URI_BASE + "/playlist?list=PL-Rk9-7qrgwd2thtw0ZUjB5HMKv2kz9Zb"
    
    def __init__(self):
        super().__init__(os.getcwd() + "\\msedgedriver.exe")
        super().get(self.URI_PLAYLIST)

        self.titles = None
        self.urls = None
        self.creators = []
        self.subscribers = []
        self.likes = []
        self.views = []

    def _stat_exchanger(self, c_unit: str) -> int:
        ABBR = {"k": 3, "m": 6, "b": 9, "t": 12, '천': 3, "만": 4, "억": 8, "조": 12}

        if c_unit.isnumeric():
            return int(c_unit)
        elif c_unit[-1].lower() in ABBR.keys():
            return int(float(c_unit[:-1]) * (10 ** ABBR[c_unit[-1].lower()]))
        else:
            return "Unit not found"

    def get_result(self, wait_time = 10):

        SEL_CONTENT = "#meta > h3 a"
        PATH_TITLE = "#title > h1"
        PATH_CREATOR = "#text > a"
        PATH_SUB_COUNT = "#owner-sub-count"
        PATH_LIKE = "#segmented-like-button > ytd-toggle-button-renderer > yt-button-shape > button > div.cbox.yt-spec-button-shape-next--button-text-content > span"
        PATH_VIEW = "#info > span:nth-child(1)"

        soup = BeautifulSoup(super().page_source, 'lxml')

        e_content = soup.select(SEL_CONTENT)

        self.titles = [c.text.strip("\n ") for c in e_content[:-1]]
        self.urls = [self.URI_BASE + c['href'] for c in e_content[:-1]]
        
        wait = WebDriverWait(super(), wait_time)

        for url in self.urls:
            super().get(url)  
  
            creators = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, PATH_CREATOR))).text
            
            subscribers = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, PATH_SUB_COUNT))).text
            subscribers = subscribers.strip("구독자 명")
            subscribers = self._stat_exchanger(subscribers)
            
            likes = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, PATH_LIKE))).text
            likes = self._stat_exchanger(likes)

            views = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, PATH_VIEW))).text
            views = views.strip("조회수 회")
            views = self._stat_exchanger(views)

            self.creators.append(creators)
            self.subscribers.append(subscribers)
            self.likes.append(likes)
            self.views.append(views)

    def print_result(self):
        if self.titles is not None:
            for i in range(len(self.titles)):
                print("title: {},\nurl: {},\nname: {},\nsub_count: {},\nlikes: {},\nviews: {}\n".format(self.titles[i], self.urls[i], self.creators[i], self.subscribers[i], self.likes[i], self.views[i]))

    def save_xlsx(self, filename):
        if self.titles is not None:
            wb = Workbook()
            ws = wb.active

            column_names = ['Title', 'URL', 'Creator Name', 'Subscribers', 'Likes', 'Vies']

            for j, col in enumerate(column_names):
                ws.cell(row = 1, column = j + 1, value = col)

            for i in range(len(self.titles)):
                ws.cell(row = 2 + i, column = 1, value = self.titles[i])
                ws.cell(row = 2 + i, column = 2, value = self.urls[i])
                ws.cell(row = 2 + i, column = 3, value = self.creators[i])
                ws.cell(row = 2 + i, column = 4, value = self.subscribers[i])
                ws.cell(row = 2 + i, column = 5, value = self.likes[i])
                ws.cell(row = 2 + i, column = 6, value = self.views[i])

            wb.save('./' + filename)
            wb.close()

In [27]:
yt_agent = YoutubePlaylist()

C:\Users\user\AppData\Local\Temp\ipykernel_3776\758002284.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  super().__init__(os.getcwd() + "\\msedgedriver.exe")


In [28]:
yt_agent.get_result()
yt_agent.close()

In [29]:
yt_agent.print_result()

title: chatGPT는 엄준식 프로그래밍 언어도 잘할까,
url: https://www.youtube.com/watch?v=SdDs0ScOFSU&list=PL-Rk9-7qrgwd2thtw0ZUjB5HMKv2kz9Zb&index=1,
name: 코딩애플,
sub_count: 135000,
likes: 2400,
views: 60000

title: 미친 파이썬 프레임워크가 나타났다.,
url: https://www.youtube.com/watch?v=47BL6WLZJ1g&list=PL-Rk9-7qrgwd2thtw0ZUjB5HMKv2kz9Zb&index=2,
name: 노마드 코더 Nomad Coders,
sub_count: 452000,
likes: 2600,
views: 120000

title: 코딩 빠르게 하는 꿀팁 및 설정 🍯 (자동 완성 기능),
url: https://www.youtube.com/watch?v=umeqCopb96w&list=PL-Rk9-7qrgwd2thtw0ZUjB5HMKv2kz9Zb&index=3,
name: 드림코딩,
sub_count: 161000,
likes: 687,
views: 18000

title: 위기,
url: https://www.youtube.com/watch?v=DEZltZdk-o0&list=PL-Rk9-7qrgwd2thtw0ZUjB5HMKv2kz9Zb&index=4,
name: 수빙수tv sooBingsoo,
sub_count: 1190000,
likes: 4100,
views: 270000

title: 한국에 집을 사는 대신, 뉴욕에서 월세살이를 시작한 부부 ep.1,
url: https://www.youtube.com/watch?v=aX_Oan6KkFw&list=PL-Rk9-7qrgwd2thtw0ZUjB5HMKv2kz9Zb&index=5,
name: 유랑쓰 YOURANGSS,
sub_count: 123000,
likes: 6600,
views: 300000



In [30]:
yt_agent.save_xlsx("result_oop.xlsx")